In [11]:
import yfinance as yf
import pandas as pd

tickers = [
    "^FCHI",
    "AB.PA",
    "A2A.MI",
    "ABCA.PA",
    "ABNX.PA",
    "ABVX.PA",
    "AC.PA",
    "ADP.PA",
    "AIR.PA",
    "AF.PA",
    "AI.PA",
    "AKW.PA",
    "ALD.PA",
    "BNP.PA",
    "BOL.PA",
    "BON.PA",
    "EN.PA",
    "BVI.PA"]
start_date = '2021-01-04'
end_date = '2022-12-30'

In [12]:
# Download stock prices
stock_data = yf.download(tickers, start=start_date, end=end_date)

# Select 'Close' prices for each stock
close_prices = stock_data['Close']

# Transform stock prices to a dataframe
df = pd.DataFrame(close_prices)

[*********************100%***********************]  18 of 18 completed


In [13]:
results = []

# Repeat for each period of watch_days
for watch_days in range(1, 201, 20):  # watch_days represents the number of days for price increase calculation
    for num_stocks_to_buy in range(3, 6):  # num_stocks_to_buy represents the number of stocks to buy
        # Calculate the number of iterations
        num_iterations = (len(df) - watch_days + 1) // watch_days

        # Perform iterations
        for i in range(num_iterations):
            # Calculate the price increase in the last watch_days days for each stock
            price_increase = df[i * watch_days:(i * watch_days) + watch_days].pct_change(watch_days).tail(1)

            # Select the num_stocks_to_buy stocks with the highest price increase
            top_stocks = price_increase.squeeze().nlargest(num_stocks_to_buy).index

            # Repeat for each period of watch_days plus hold_days
            for hold_days in range(1, 201, 50):  # hold_days represents the number of additional days before selling
                for max_loss in range(0, 201, 40):  # max_loss represents the percentage decrease threshold for selling
                    # Calculate the buying prices at the start of the hold period
                    buy_prices = df.loc[df.index[(i * watch_days) + watch_days - 1], top_stocks]

                    # Calculate the selling prices after watch_days + hold_days days
                    sell_prices = df.shift(watch_days + hold_days).loc[df.index[(i * watch_days) + watch_days - 1], top_stocks]

                    # Calculate the profit for each stock
                    profit = (sell_prices / buy_prices - 1) * 100

                    # Filter the stocks to sell based on decrease threshold max_loss
                    sell_mask = (sell_prices / buy_prices - 1) < -max_loss / 100
                    profit[sell_mask] = -max_loss  # Set profit to -max_loss for stocks that decrease beyond threshold

                    # Calculate the total profit for the selected stocks
                    total_profit = profit.sum()

                    results.append({'watch_days': watch_days, 'num_stocks_to_buy': num_stocks_to_buy,
                                    'hold_days': hold_days, 'max_loss': max_loss,
                                    'profit': total_profit})

results_df = pd.DataFrame(results)

In [15]:
results_df.sort_values("profit", ascending=False, inplace=True)
print(results_df.to_markdown())

|       |   watch_days |   num_stocks_to_buy |   hold_days |   max_loss |        profit |
|------:|-------------:|--------------------:|------------:|-----------:|--------------:|
| 37654 |           51 |                   5 |         151 |        160 |  465.337      |
| 37655 |           51 |                   5 |         151 |        200 |  465.337      |
| 37652 |           51 |                   5 |         151 |         80 |  465.337      |
| 37651 |           51 |                   5 |         151 |         40 |  465.337      |
| 37650 |           51 |                   5 |         151 |          0 |  465.337      |
| 37653 |           51 |                   5 |         151 |        120 |  465.337      |
| 34482 |            1 |                   5 |         151 |          0 |  430.854      |
| 34487 |            1 |                   5 |         151 |        200 |  430.854      |
| 34486 |            1 |                   5 |         151 |        160 |  430.854      |
| 34485 | 